In [1]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

# Seeding
os.environ['PYTHONHASHSEED'] = '42'
np.random.seed(42)
tf.random.set_seed(42)

# Path
dataset_path = "D:/cv/newCvcp/dataset/nonaug/test"
save_path = "D:/cv/newCvcp/prediction/non-aug"
model_path = "D:/cv/newCvcp/New folder/unet-non-aug.keras"

# Create folder
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Folder to save predicted mask
create_dir(save_path)

# Load model
model = tf.keras.models.load_model(model_path)
model.summary()

test_x = sorted(glob(os.path.join(dataset_path, "images", "*")))
print(f"Test Images: {len(test_x)}")

time_taken = []

for x in tqdm(test_x):
    name = os.path.basename(x)  # Extract the filename from the path
    
    # Read the image using OpenCV with color channels
    img = cv2.imread(x, cv2.IMREAD_COLOR)
    
    # Normalize the image data
    img = img / 255.0
    
    # Add a batch dimension to the image for model prediction
    img = np.expand_dims(img, axis=0)
    
    start_time = time.time()  # Record the start time before making the prediction
    
    pred_mask = model.predict(img)  # Use the model to predict the mask for the input image
    
    total_time = time.time() - start_time  # Calculate the total time taken for prediction
    time_taken.append(total_time)  # Append the total time taken to the time_taken list
    
    # Threshold the predicted mask
    pred_mask = (pred_mask > 0.5).astype(np.uint8) * 255
    
    # Convert the predicted mask to the same shape as the original image
    pred_mask = cv2.resize(pred_mask[0], (img.shape[2], img.shape[1]), interpolation=cv2.INTER_NEAREST)
    
    # Overlay the mask on the original image for visualization
    overlay = cv2.addWeighted(img[0].astype(np.uint8), 0.7, cv2.cvtColor(pred_mask, cv2.COLOR_GRAY2BGR).astype(np.uint8), 0.3, 0)
    
    # Save the overlay image
    cv2.imwrite(os.path.join(save_path, f"{os.path.splitext(name)[0]}_segmented.jpg"), overlay)

# Calculate mean time per inference and mean FPS
mean_time = np.mean(time_taken)
mean_fps = 1 / mean_time

print(f"Mean Time: {mean_time:.5f} seconds")
print(f"Mean FPS: {mean_fps:.5f}")


Model: "UNET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]  

 Total params: 93,142,341 (355.31 MB)

 Trainable params: 31,043,521 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


 95%|█████████▌| 21/22 [00:33<00:01,  1.50s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


100%|██████████| 22/22 [00:35<00:00,  1.61s/it]

Mean Time: 1.59974 seconds
Mean FPS: 0.62510
